## AP1 - Pose Estimation - Evaluation

### 1. Extract MP4 from MKV-Files:


In [7]:
from Utils import extractMp4

In [8]:
extractMp4()

### 2. Run Mediapipe Pose Estimation on MP4 File

In [10]:
from PoseEstimation import mediapipe_estimation

In [11]:
mediapipe_estimation(input_mp4='1_output_extract', output_xlsx='2_output_mediapipe', verbose=False)


### 3. Create Transformed Depth, read depth, create 3D-coordinates and store new xlsx-File

In [3]:
from PoseEstimation import depth_3d_estimation, depth_3d_estimation_evaluation

If marker is visible in the video, take the normal estimation function.
If not, take the evaluation function. The rotation matrix is then not calculated with the marker, but a default rotation matrix is loaded.

In [13]:
depth_3d_estimation(input_mkv='1_output_extract', input_xlsx='2_output_mediapipe', output_xlsx='3_output_depth3d', interpolation=True, verbose=False, switch_axes=False)

### Struktur Koordinaten je Landmark am Ende

| MP_x | MP_y | MP_z | vis  | Depth (Kinect Wert in mm) | 3D_x | 3D_y | World_x | World_y | World_z | Interpolated-Flag |
|------|------|------|-------------------------------|---|------|------|---------|---------|---------|---|
|      |      |      |                               |  |      |      |         |         |         | |

## 4. Evaluation


Ausrichtung der Puppe

In [1]:
import math
import numpy as np
import cv2
import pyk4a
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import time
import imutils
from helpers import colorize, convert_to_bgra_if_required
from pyk4a import PyK4APlayback
import pyk4a
from WorldCoordinates import calculateWorldTransMat
from Utils import getAllMp4, getAllMkv
import os
from statistics import median
#
# k4a = pyk4a.PyK4A(
#     pyk4a.Config(
#         color_resolution=pyk4a.ColorResolution.RES_1080P,
#         depth_mode=pyk4a.DepthMode.WFOV_UNBINNED,
#         synchronized_images_only=False,
#         camera_fps=pyk4a.FPS.FPS_15
#     ),
# )
#
# k4a.start()
#
# intrinsic = k4a.calibration.get_camera_matrix(pyk4a.calibration.CalibrationType.COLOR)
# distortion = k4a.calibration.get_distortion_coefficients(pyk4a.calibration.CalibrationType.COLOR)
# k4a.stop()
# print(intrinsic)
# print(distortion)


ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

def aruco_display(image, corners, ids, rejected):

	if len(corners) > 0:

		ids = ids.flatten()

		for (markerCorner, markerID) in zip(corners, ids):

			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners

			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)

			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)

			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))

	return image


def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #aruco_dict = aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()
    trans_mat = 0
    # Für diesen Frame, detektiere alle Marker und speichere ihre Eckpunkte und IDs ab
    total_corners, total_ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict_type,parameters=parameters)
    # Wenn Eckpunkte, also Marker vorhanden sind
    if len(total_corners) > 0:
        # Berechne rotations und translation vectors für alle Marker
        rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(total_corners, 0.17, matrix_coefficients,
                                                                       distortion_coefficients)
        total_markers = range(0, total_ids.size)
        for id, corners, i in zip(total_ids, total_corners, total_markers):
            print(f"rvec: {rvec}, tvec: {tvec}")
            print(corners)
            #total_markers = range(0, ids.size)
            #cv2.polylines(frame, [corners.astype(np.int32)], True, (0, 255, 255), 4, cv2.LINE_AA)
            corners = corners.reshape(4, 2)
            #corners = np.asarray(corners)
            corners = corners.astype(int)
            corners = corners.reshape((4,2))
            top_right = corners[0].ravel()
            bottom_right = corners[1].ravel()
            bottom_left = corners[2].ravel()
            top_left = corners[3].ravel()
            print(top_right)
            distance = np.sqrt(tvec[i][0][2] ** 2 + tvec[i][0][0] ** 2 + tvec[i][0][1] ** 2 )

            #cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec[i], tvec[i], 3, 2)
            # cv2.putText(frame, f"id: {id[0]} Distance: {round(distance, 3)}", top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)


        ## CREATE TRANSFORMATION MATRIX FROM ROTATION AND TRANSLATION VECTORS
        # convert rvec to rotation matrix
            rmat, _ = cv2.Rodrigues(rvec[i])
            print(rmat@rmat.T)
            print(rmat.T)
        # create a 4x4 transformation matrix with translation vector
            trans_mat = np.eye(4)
            trans_mat[:3, :3] = rmat
            trans_mat[:3, 3] = tvec[i].ravel()
            print(trans_mat)
    return frame, trans_mat


# LIVE VERSION ZUM ANZEIGEN DES WELTKOODINATENSYSTEMS

In [2]:
import imutils

aruco_type = "DICT_5X5_250"
frame_nr = 0

arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_250)

arucoParams = cv2.aruco.DetectorParameters_create()

k4a = pyk4a.PyK4A(
    pyk4a.Config(
        color_resolution=pyk4a.ColorResolution.RES_2160P,
        synchronized_images_only=False,
        camera_fps=pyk4a.FPS.FPS_30
    ),
)

k4a.open()
calibration = k4a.calibration
mp_drawing = mp.solutions.drawing_utils
# mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Store camera parameters
intrinsic_matrix = calibration.get_camera_matrix(pyk4a.calibration.CalibrationType.COLOR)
distortion_vector = calibration.get_distortion_coefficients(pyk4a.calibration.CalibrationType.COLOR)
k4a.start()
count = 0
while True:

	with mp_pose.Pose(min_detection_confidence=0.75,
				  min_tracking_confidence=0.5, model_complexity=2, static_image_mode=True) as pose:
		try:
			capture = k4a.get_capture()
			if np.any(capture.color):
				img = capture.color[:, :, :3]
				img = np.ascontiguousarray(img, dtype=np.uint8)
				image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
				# Performance optimization for mediapipe processing
				image.flags.writeable = False
				# Process image with mediapipe holistic model to detect landmarks
				results = pose.process(image)
				# Undo writeable flag
				image.flags.writeable = True
				# convert back to BGR for CV2 functions
				image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

				# draw pose landmarks
				mp_drawing.draw_landmarks(
					image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

				markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(img, dictionary=arucoDict, parameters=arucoParams)
				image = aruco_display(image, markerCorners, markerIds, rejectedCandidates)
				image, trans_mat = pose_estimation(image, arucoDict, intrinsic_matrix, distortion_vector)
				image_copy = imutils.resize(image, width=720)
				rotated_copy = cv2.rotate(image_copy, cv2.ROTATE_90_COUNTERCLOCKWISE)
				rotated_original = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
				cv2.imshow('Estimated Pose', rotated_copy)
				# cv2.imwrite("results/evaluation/final_frames_LIGHTS_ON_NearPos_mp%d.jpg" % count, rotated_original)
				count += 1
			key = cv2.waitKey(1) & 0xFF
			if key == ord('q'):
				break
		except EOFError:
			break

#cap.release()
cv2.destroyAllWindows()
k4a.stop()

[Inference] ArUco marker ID: 0
rvec: [[[-0.29482017  2.11048372 -0.2397925 ]]], tvec: [[[-1.0350281  -0.43176809  1.96316332]]]
[[[ 985.  807.]
  [ 824.  730.]
  [ 870.  582.]
  [1025.  665.]]]
[985 807]
[[1.00000000e+00 5.88910880e-17 5.00356307e-18]
 [5.88910880e-17 1.00000000e+00 9.95908862e-17]
 [5.00356307e-18 9.95908862e-17 1.00000000e+00]]
[[-0.51352514 -0.30265735 -0.80292618]
 [-0.11481162  0.95155148 -0.28525088]
 [ 0.85035888 -0.05429824 -0.52339419]]
[[-0.51352514 -0.11481162  0.85035888 -1.0350281 ]
 [-0.30265735  0.95155148 -0.05429824 -0.43176809]
 [-0.80292618 -0.28525088 -0.52339419  1.96316332]
 [ 0.          0.          0.          1.        ]]
[Inference] ArUco marker ID: 0
rvec: [[[-0.29648608  2.09415432 -0.21783362]]], tvec: [[[-1.02725392 -0.42859552  1.94898498]]]
[[[ 985.  808.]
  [ 824.  731.]
  [ 871.  581.]
  [1025.  664.]]]
[985 808]
[[ 1.00000000e+00 -1.67908038e-16 -6.75904630e-17]
 [-1.67908038e-16  1.00000000e+00 -1.71428932e-16]
 [-6.75904630e-17 -1.7

# Evaluation Report Funktion
## 4 Säulen der Evaluation:
### 1. Distanzen von Landmarks zueinander
### 2. Winkel zwischen Landmarks
### 3. Rotation des Kopfes
### 4. Konsistenz der anatomisch unveränderlichen Landmarks

In [1]:
from FeatureExtraction import euclideanDistance3dCalc, euclideanDistance3dCalcToOrigin
from Utils import parsXlsxToDf
import numpy as np
import pandas as pd

In [2]:
measurement_connections = [["RIGHT_FOOT_INDEX", "RIGHT_KNEE"], ["RIGHT_FOOT_INDEX", "LEFT_KNEE"], ["RIGHT_FOOT_INDEX", "RIGHT_HIP"], ["RIGHT_FOOT_INDEX", "LEFT_HIP"],
						   ["RIGHT_FOOT_INDEX", "RIGHT_WRIST"], ["RIGHT_FOOT_INDEX", "LEFT_WRIST"], ["RIGHT_FOOT_INDEX", "RIGHT_SHOULDER"], ["RIGHT_FOOT_INDEX", "LEFT_SHOULDER"], ["RIGHT_FOOT_INDEX", "NOSE"],
						   ["LEFT_FOOT_INDEX", "RIGHT_KNEE"], ["LEFT_FOOT_INDEX", "LEFT_KNEE"], ["LEFT_FOOT_INDEX", "RIGHT_HIP"], ["LEFT_FOOT_INDEX", "LEFT_HIP"],
						   ["LEFT_FOOT_INDEX", "RIGHT_WRIST"], ["LEFT_FOOT_INDEX", "LEFT_WRIST"], ["LEFT_FOOT_INDEX", "RIGHT_SHOULDER"], ["LEFT_FOOT_INDEX", "LEFT_SHOULDER"], ["LEFT_FOOT_INDEX", "NOSE"],
						   ["RIGHT_SHOULDER", "LEFT_SHOULDER"], ["RIGHT_WRIST", "LEFT_WRIST"], ["RIGHT_HIP", "RIGHT_SHOULDER"], ["LEFT_HIP", "LEFT_SHOULDER"],["LEFT_HIP", "RIGHT_HIP"], ["RIGHT_HIP", "LEFT_SHOULDER"],
						   ["RIGHT_SHOULDER", "NOSE"], ["LEFT_SHOULDER", "NOSE"], ["MOUTH_LEFT", "MOUTH_RIGHT"], ["RIGHT_THUMB", "LEFT_THUMB"], ["RIGHT_FOOT_INDEX", "LEFT_FOOT_INDEX"]]

In [3]:
file1 = "5_storage/Evaluation_Final/MitLichtRear/20230629171406135_3_MESSUNG1_001_3d_interpolated.xlsx"
file2 = "5_storage/Evaluation_Final/MitLichtRear/20230629171406135_3_MESSUNG2_001_3d_interpolated.xlsx"
df_eval_measurement_1 = parsXlsxToDf(file1)
df_eval_measurement_2 = parsXlsxToDf(file2)

In [ ]:

measurement_id_1 = file1.split("/")[3]
measurement_id_1 = measurement_id_1[:-5]
measurement_id_2 = file2.split("/")[3]
measurement_id_2 = measurement_id_2[:-5]

df_report = pd.DataFrame()
for i in measurement_connections:
	dist_1 = euclideanDistance3dCalc(lm1=i[0], lm2=i[1], onlyDistSeriesReturn=True, df_in=df_eval_measurement_1)
	dist_2 = euclideanDistance3dCalc(lm1=i[0], lm2=i[1], onlyDistSeriesReturn=True, df_in=df_eval_measurement_2)
	df_report = df_report.append({"Messung": measurement_id_1, "Verbindung": str(i[0]) + " - " + str(i[1]), "Average": np.average(dist_1), "Median": np.median(dist_1)}, ignore_index=True)
	df_report = df_report.append({"Messung": measurement_id_2, "Verbindung": str(i[0]) + " - " + str(i[1]), "Average": np.average(dist_2), "Median": np.median(dist_2)}, ignore_index=True)
df_report.to_excel("results/evaluation/evaluation_report.xlsx")

In [ ]:
# Für alle

## Kopfrotation

In [15]:
from FeatureExtraction import calculate_angle_between_vectors, calculate_head_rotation, calculate_head_rotation_euler
from Utils import parsXlsxToDf
import numpy as np
import pandas as pd
import os
import warnings

In [22]:
directory = "5_storage/Evaluation_Final/Kopfdrehung/"

In [ ]:
# Disable FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Create an empty DataFrame
df_report = pd.DataFrame(columns=["Measurement", "Average", "Median", "Angles"])
for subdir, dirs, files in os.walk(directory):
	for file in files:
		if file.endswith("_interpolated.xlsx"):
			file = os.path.join(subdir, file)
			df_eval_head = parsXlsxToDf(file)
			anglesHeadShoulder = calculate_angle_between_vectors(line1_lm1="LEFT_SHOULDER", line1_lm2="RIGHT_SHOULDER", line2_lm1="LEFT_EAR", line2_lm2="RIGHT_EAR", df_in=df_eval_head)
            # Extract the measurement name from the file path
			measurement_name = file.split("_")[-4]
			df_report = df_report.append({"Measurement": measurement_name, "Average": np.average(anglesHeadShoulder),"Median": np.median(anglesHeadShoulder), "Angles": anglesHeadShoulder}, ignore_index=True)

df_report.to_excel("results/evaluation/evaluation_report_shoulders_ears.xlsx")
print(df_report)

In [23]:
fileHead = "5_storage/Evaluation_Final/Kopfdrehung/60l/20230620171406135_3_Kopf60l1_001_3d_interpolated.xlsx"
df_eval_head = parsXlsxToDf(fileHead)

In [24]:
anglesHeadShoulder = calculate_angle_between_vectors(line1_lm1="LEFT_SHOULDER", line1_lm2="RIGHT_SHOULDER", line2_lm1="LEFT_EYE", line2_lm2="RIGHT_EYE", df_in=df_eval_head)

In [25]:
np.average(anglesHeadShoulder)

86.26171111612278

In [26]:
np.median(anglesHeadShoulder)

86.21734924109623

In [10]:
calculate_head_rotation(lm1="NOSE", lm2="LEFT_SHOULDER", lm3="RIGHT_SHOULDER", df_in=df_eval_head)

array([-74.50913169, -70.701294  , -77.64397384, -70.08442   ,
       -87.51805192, -84.41627703, -77.49612012, -77.20652793,
       -72.87723974, -74.46293452, -90.61152402, -75.87911044,
       -86.95206425, -77.19694668, -81.2744398 , -81.25732703,
       -80.1401535 , -75.56342854, -69.37208052, -81.63145956,
       -81.49826416, -73.62025037, -74.06368282, -73.5623884 ,
       -76.72913162, -72.74799068, -79.49730327, -76.71906012,
       -80.57823919, -65.32458648, -79.81036652, -72.55287625,
       -70.49490707, -80.32042434, -79.40665493, -71.37645798,
       -83.33492796, -82.90105563, -76.36860475, -71.62573841,
       -76.35674519, -72.755865  , -77.77181662, -71.79166402,
       -70.97923145, -90.50767276, -75.11899321, -64.90200265,
       -69.60309485, -83.08874001])

In [274]:
landmark1 = 'NOSE'
landmark2 = 'LEFT_EYE'
landmark3 = 'RIGHT_EYE'

yaw,pitch, roll = calculate_head_rotation_euler(landmark1, landmark2, landmark3, df_in=df_eval_head)
print("Yaw angle:", yaw)
print("Pitch angle:", pitch)
print("Roll angle:", roll)

Yaw angle: [127.71856175 136.16208794 137.20935431 139.41211856 132.2435505
 132.2435505  131.47411033 136.98461637 135.11652225 133.00170116
 130.58710418 132.81896318 133.76660399 125.82781856 128.70938024
 132.80539477 135.54201927 135.54491271 135.54683741 133.53491256
 131.26352085 134.89747458 136.98437664 133.00170116 132.23705275
 133.00170116 133.73176407 135.1081784  132.22271589 137.53301554
 133.5336903  131.45557788 133.73763389 130.61252916 132.99309804
 135.45590446 129.662488   131.29146159 133.49333773 136.06124174
 136.86305633 131.31977706 132.79190125 135.45659823 134.17682468
 132.0527944  130.52139076 136.06237363 134.18092234 131.29120617]
Pitch angle: [64.51238622 53.23544239 50.48700909 49.79578028 61.91955852 61.91955852
 62.93223939 53.55068075 57.33006097 60.80226251 63.99001825 59.42254861
 59.91208446 67.48239739 64.73876495 59.44368916 54.55305474 54.54708725
 54.54312523 58.26808253 61.67307866 55.83000017 53.55186793 60.80226251
 61.89115819 60.80226251

In [3]:
from FeatureExtraction import calculate_head_rotation_euler2

In [5]:
yaw, pitch, roll = calculate_head_rotation_euler2(df_in=df_eval_head)
print("Yaw angle:", yaw)
print("Pitch angle:", pitch)
print("Roll angle:", roll)

Yaw angle: [ 104.52831201 -134.87589117   41.90776362]
Pitch angle: [-19.38205967 -11.50435501 -27.0956887 ]
Roll angle: [-129.12295834 -137.96480173 -143.96796538]


## Anatomische Distanzen in Bewegung
zB Oberarmlänge in Bewegung sollte konstant sein
Daten: Alle BonaFide Durchgänge?
Folgende Punkte aufzeigen:
- Beispiele sind Oberarmlänge, Oberschenkellänge, Schulterabstand und Hüftabstand
- Ein mal graphisch darstellen
- Ein mal als Tabelle mit Vergleich interpoliert und nicht interpoliert

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Utils import parsXlsxToDf
from FeatureExtraction import euclideanDistance3dCalc, angleThreePoints3dCalc

In [10]:
landmark_pairs = [
    ["RIGHT_WRIST", "RIGHT_PINKY"],
    ["LEFT_WRIST", "LEFT_PINKY"],
    ["RIGHT_SHOULDER", "RIGHT_ELBOW"],
    ["LEFT_SHOULDER", "LEFT_ELBOW"],
    ["RIGHT_SHOULDER", "LEFT_SHOULDER"],
    ["RIGHT_HIP", "LEFT_HIP"],
    ["RIGHT_HIP", "RIGHT_KNEE"],
    ["LEFT_HIP", "LEFT_KNEE"],
    ["RIGHT_KNEE", "RIGHT_ANKLE"],
    ["LEFT_KNEE", "LEFT_ANKLE"]
]
for joint in landmark_pairs:
	print(joint[0] + "_" + joint[1])

RIGHT_WRIST_RIGHT_PINKY
LEFT_WRIST_LEFT_PINKY
RIGHT_SHOULDER_RIGHT_ELBOW
LEFT_SHOULDER_LEFT_ELBOW
RIGHT_SHOULDER_LEFT_SHOULDER
RIGHT_HIP_LEFT_HIP
RIGHT_HIP_RIGHT_KNEE
LEFT_HIP_LEFT_KNEE
RIGHT_KNEE_RIGHT_ANKLE
LEFT_KNEE_LEFT_ANKLE


In [12]:
df_movement_interpolated = parsXlsxToDf("5_storage/BonaFide/interpolated/20230509155025930_3_PRJSAYBY_001_3d_interpolated.xlsx")
df_movement = parsXlsxToDf("5_storage/BonaFide/original/20230509155025930_3_PRJSAYBY_001_3d.xlsx")

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Utils import parsXlsxToDf
from FeatureExtraction import euclideanDistance3dCalc, angleThreePoints3dCalc

# Define landmark pairs
landmark_pairs = [
    ["RIGHT_WRIST", "RIGHT_PINKY"],
    ["LEFT_WRIST", "LEFT_PINKY"],
    ["RIGHT_SHOULDER", "RIGHT_ELBOW"],
    ["LEFT_SHOULDER", "LEFT_ELBOW"],
    ["RIGHT_SHOULDER", "LEFT_SHOULDER"],
    ["RIGHT_HIP", "LEFT_HIP"],
    ["RIGHT_HIP", "RIGHT_KNEE"],
    ["LEFT_HIP", "LEFT_KNEE"],
    ["RIGHT_KNEE", "RIGHT_ANKLE"],
    ["LEFT_KNEE", "LEFT_ANKLE"]
]

# Load dataframes
df_movement_interpolated = parsXlsxToDf("5_storage/BonaFide/interpolated/20230509155025930_3_PRJSAYBY_001_3d_interpolated.xlsx")
df_movement = parsXlsxToDf("5_storage/BonaFide/original/20230509155025930_3_PRJSAYBY_001_3d.xlsx")

# Calculate and append Euclidean distances with standard deviation and relative standard deviation
for joint in landmark_pairs:
    print(joint[0] + "_" + joint[1])
    df_movement_interpolated = euclideanDistance3dCalc(df_in=df_movement_interpolated, lm1=joint[0], lm2=joint[1], outlierCorrection=False)
    distances_interpolated = df_movement_interpolated[joint[0] + "_" + joint[1] + "_euclDist"]
    std_interpolated = np.std(distances_interpolated)
    median_interpolated = np.median(distances_interpolated)
    df_movement_interpolated[joint[0] + "_" + joint[1] + "_std"] = std_interpolated
    df_movement_interpolated[joint[0] + "_" + joint[1] + "_RSD"] = (std_interpolated / median_interpolated) * 100

    df_movement = euclideanDistance3dCalc(df_in=df_movement, lm1=joint[0], lm2=joint[1], outlierCorrection=False)
    distances = df_movement[joint[0] + "_" + joint[1] + "_euclDist"]
    std = np.std(distances)
    median = np.median(distances)
    df_movement[joint[0] + "_" + joint[1] + "_std"] = std
    df_movement[joint[0] + "_" + joint[1] + "_RSD"] = (std / median) * 100

# Save both dataframes as xlsx under the results folder
df_movement_interpolated.to_excel("results/evaluation/interpolated-joints.xlsx")
df_movement.to_excel("results/evaluation/non-interpolated-joints.xlsx")


RIGHT_WRIST_RIGHT_PINKY
Done
Done
LEFT_WRIST_LEFT_PINKY
Done
Done
RIGHT_SHOULDER_RIGHT_ELBOW
Done
Done
LEFT_SHOULDER_LEFT_ELBOW
Done
Done
RIGHT_SHOULDER_LEFT_SHOULDER
Done
Done
RIGHT_HIP_LEFT_HIP
Done
Done
RIGHT_HIP_RIGHT_KNEE
Done
Done
LEFT_HIP_LEFT_KNEE
Done
Done
RIGHT_KNEE_RIGHT_ANKLE
Done
Done
LEFT_KNEE_LEFT_ANKLE
Done
Done


In [16]:
import os
import numpy as np
import pandas as pd
from Utils import parsXlsxToDf
from FeatureExtraction import euclideanDistance3dCalc, angleThreePoints3dCalc

# Define landmark pairs
landmark_pairs = [
    ["RIGHT_WRIST", "RIGHT_PINKY"],
    ["LEFT_WRIST", "LEFT_PINKY"],
    ["RIGHT_SHOULDER", "RIGHT_ELBOW"],
    ["LEFT_SHOULDER", "LEFT_ELBOW"],
    ["RIGHT_SHOULDER", "LEFT_SHOULDER"],
    ["RIGHT_HIP", "LEFT_HIP"],
    ["RIGHT_HIP", "RIGHT_KNEE"],
    ["LEFT_HIP", "LEFT_KNEE"],
    ["RIGHT_KNEE", "RIGHT_ANKLE"],
    ["LEFT_KNEE", "LEFT_ANKLE"]
]

# Directory path
directory = r"C:\Users\Nico\PycharmProjects\pythonProject2\5_storage\BonaFide\interpolated"

# Iterate through all Excel files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(directory, filename)

        # Load dataframe from Excel file
        df_movement_interpolated = parsXlsxToDf(file_path)

        # Calculate and append Euclidean distances with standard deviation and relative standard deviation
        for joint in landmark_pairs:
            df_movement_interpolated = euclideanDistance3dCalc(df_in=df_movement_interpolated, lm1=joint[0], lm2=joint[1], outlierCorrection=False)
            distances_interpolated = df_movement_interpolated[joint[0] + "_" + joint[1] + "_euclDist"]
            std_interpolated = np.std(distances_interpolated)
            median_interpolated = np.median(distances_interpolated)
            df_movement_interpolated[joint[0] + "_" + joint[1] + "_SD"] = std_interpolated
            df_movement_interpolated[joint[0] + "_" + joint[1] + "_RSD"] = (std_interpolated / median_interpolated) * 100

        # Save the updated dataframe with SD and RSD as a new Excel file
        result_filename = f"results/{filename[:-5]}_sd_rsd.xlsx"
        df_movement_interpolated.to_excel(result_filename, index=False)

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [19]:
import os
import numpy as np
import pandas as pd
from Utils import parsXlsxToDf
from FeatureExtraction import euclideanDistance3dCalc, angleThreePoints3dCalc

# Define landmark pairs
landmark_pairs = [
    ("RIGHT_WRIST", "RIGHT_PINKY"),
    ("LEFT_WRIST", "LEFT_PINKY"),
    ("RIGHT_SHOULDER", "RIGHT_ELBOW"),
    ("LEFT_SHOULDER", "LEFT_ELBOW"),
    ("RIGHT_SHOULDER", "LEFT_SHOULDER"),
    ("RIGHT_HIP", "LEFT_HIP"),
    ("RIGHT_HIP", "RIGHT_KNEE"),
    ("LEFT_HIP", "LEFT_KNEE"),
    ("RIGHT_KNEE", "RIGHT_ANKLE"),
    ("LEFT_KNEE", "LEFT_ANKLE")
]

# Directory path
directory ="5_storage\BonaFide\original"

# Initialize a dictionary to store SD and RSD values
sd_rsd_dict = {}

# Iterate through all Excel files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(directory, filename)

        # Load dataframe from Excel file
        df_movement_interpolated = parsXlsxToDf(file_path)

        # Calculate and append Euclidean distances with standard deviation
        for joint in landmark_pairs:
            df_movement_interpolated = euclideanDistance3dCalc(df_in=df_movement_interpolated, lm1=joint[0], lm2=joint[1], outlierCorrection=False)
            distances_interpolated = df_movement_interpolated[joint[0] + "_" + joint[1] + "_euclDist"]
            std_interpolated = np.std(distances_interpolated)

            # Store SD values in the dictionary
            if joint not in sd_rsd_dict:
                sd_rsd_dict[joint] = [std_interpolated]
            else:
                sd_rsd_dict[joint].append(std_interpolated)

# Calculate mean SD and RSD across all Excel files
mean_sd_rsd_dict = {}
for joint, sd_list in sd_rsd_dict.items():
    mean_sd = np.mean(sd_list)
    median_sd = np.median(sd_list)
    mean_rsd = (mean_sd / median_sd) * 100
    mean_sd_rsd_dict[joint] = [mean_sd, mean_rsd]

# Create dataframe from mean SD and RSD dictionary
df_mean_sd_rsd = pd.DataFrame.from_dict(mean_sd_rsd_dict, orient='index', columns=['Mean SD', 'Mean RSD'])

# Save the mean SD and RSD dataframe as an Excel file
result_filename = "results/mean_sd_rsd.xlsx"
df_mean_sd_rsd.to_excel(result_filename)


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
